# MLB Predictor Project

Group 21, Plotters for Success

Gerardo Skrut, Victor Gikunda, Mathew Huang

In [2]:
import pandas as pd
import seaborn as sn
import pickle
import chardet
import sklearn as sk
from meteostat import Point, Hourly
from datetime import datetime
import matplotlib

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Cleaning

Prior to importing the data, we cleaned and explored the existing data.

In [13]:
#batting data cleaning
# Load your CSV file into a DataFrame
df = pd.read_csv('2023batting.csv')

# Specify the columns you want to keep
columns_to_keep = ['gid', 'id', 'team', 'b_pa', 'b_ab', 'b_h', 'b_d', 'b_t', 'b_hr', 'b_rbi', 'b_w', 'b_k', 'date', 'wl']  # Replace with your column names

# Create a new DataFrame with only the selected columns
df_filtered = df[columns_to_keep]

# Optionally, save the filtered DataFrame to a new CSV
df_filtered.to_csv('2023_filtered_batting_data.csv', index=False)

batting_data = pd.read_csv('2023_filtered_batting_data.csv')
batting_data.head()

,gid,id,team,b_pa,b_ab,b_h,b_d,b_t,b_hr,b_rbi,b_w,b_k,date,wl
0,BOS202303300,mullc002,BAL,6,4,1,0,0,0,1,2,1,20230330,w
1,BOS202303300,rutsa001,BAL,6,5,5,0,0,1,4,1,0,20230330,w
2,BOS202303300,santa003,BAL,6,6,2,1,0,0,0,0,2,20230330,w
3,BOS202303300,mcker001,BAL,0,0,0,0,0,0,0,0,0,20230330,w
4,BOS202303300,mounr001,BAL,6,4,1,1,0,0,1,2,0,20230330,w


In [11]:
# Load your CSV file into a DataFrame
df = pd.read_csv('2023pitching.csv')

# Specify the columns you want to keep
columns_to_keep = ['gid', 'id', 'team', 'p_seq', 'p_h', 'p_r', 'p_er', 'p_w', 'p_hbp', 'p_wp', 'date', 'wl']  # Replace with your column names

# Create a new DataFrame with only the selected columns
df_filtered = df[columns_to_keep]

# Optionally, save the filtered DataFrame to a new CSV
df_filtered.to_csv('2023_filtered_pitching_data.csv', index=False)

pitching_data = pd.read_csv('2023_filtered_pitching_data.csv')
pitching_data.head()

In [27]:
park_data= pd.read_csv('ballparks.csv')
park_data = park_data.rename(columns={'team_name': 'site'})
park_data['site'].unique().tolist()

['ATL',
 'AZ',
 'BAL',
 'BOS',
 'CHC',
 'CIN',
 'CLE',
 'COL',
 'CWS',
 'DET',
 'HOU',
 'KC',
 'LAA',
 'LAD',
 'MIA',
 'MIL',
 'MIN',
 'NYM',
 'NYY',
 'OAK',
 'PHI',
 'PIT',
 'SD',
 'SEA',
 'SF',
 'STL',
 'TB',
 'TEX',
 'TOR',
 'WSH']

In [7]:
df = pd.read_csv('2023gameinfo.csv')
print(df.columns)
columns_to_keep = ['gid', 'attendance', 'hometeam', 'fieldcond', 'precip', 'sky', 'temp', 'winddir','windspeed', 'umplf', 'umprf']
df_filtered = df[columns_to_keep]
print(df_filtered['hometeam'].unique().tolist())

# Optionally, save the filtered DataFrame to a new CSV
df_filtered.to_csv('2023_filtered_game_data.csv', index=False)
game_data = pd.read_csv('2023_filtered_game_data.csv')
game_data.head(20)
print(game_data['winddir'].unique().tolist())

Index(['gid', 'visteam', 'hometeam', 'site', 'date', 'number', 'starttime',
       'daynight', 'innings', 'tiebreaker', 'usedh', 'htbf', 'timeofgame',
       'attendance', 'fieldcond', 'precip', 'sky', 'temp', 'winddir',
       'windspeed', 'oscorer', 'forfeit', 'umphome', 'ump1b', 'ump2b', 'ump3b',
       'umplf', 'umprf', 'wp', 'lp', 'save', 'gametype', 'vruns', 'hruns',
       'wteam', 'lteam', 'line', 'batteries', 'lineups', 'box', 'pbp',
       'season'],
      dtype='object')
['BOS', 'CHN', 'CIN', 'HOU', 'KCA', 'LAN', 'MIA', 'NYA', 'OAK', 'SDN', 'SEA', 'SLN', 'TBA', 'TEX', 'WAS', 'CHA', 'MIL', 'ARI', 'ATL', 'COL', 'DET', 'ANA', 'BAL', 'CLE', 'MIN', 'NYN', 'PHI', 'PIT', 'SFN', 'TOR', 'ALS']
['ltor', 'rtol', 'fromrf', 'unknown', 'torf', 'tolf', 'fromlf', 'tocf', 'fromcf']


In [25]:
merged_data = game_data.merge(park_data, on = 'site', how = 'left')
merged_data.head()

,gid,attendance,site,fieldcond,precip,sky,temp,winddir,umplf,umprf,...,center_field,right_field,min_wall_height,max_wall_height,hr_park_effects,extra_distance,avg_temp,elevation,roof,daytime
0,BOS202303300,36049,BOS,unknown,none,sunny,38,ltor,(none),(none),...,420.0,302.0,3.0,37.0,102.0,-4.7,69.5,21.0,0.00,0.31
1,CHN202303300,36054,CHI,unknown,none,sunny,42,rtol,(none),(none),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CIN202303300,44063,CIN,unknown,none,sunny,61,fromrf,(none),(none),...,404.0,325.0,8.0,12.0,150.0,1.6,77.9,535.0,0.00,0.50
3,HOU202303300,43032,HOU,unknown,none,dome,73,unknown,(none),(none),...,409.0,326.0,7.0,21.0,101.0,-1.3,73.0,45.0,0.99,0.32
4,KCA202303300,38351,KAN,unknown,none,cloudy,67,rtol,(none),(none),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Implementing Neural Networks

To Tune or Neural Network (NN), we are using different numbers. To do so, we will use the gridsearch CV function to process our Data 

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In this portion, we are using a pipeline to scale our neural network and testing out various hyperparameters to determine which one works best

In [ ]:
pline = Pipeline([('scaling', sk.preprocessing.StandardScaler()), ('pca', sk.decomposition.PCA()),
                  ('nnet', MLPClassifier())])

param_grid = {
    'pca__n_components':list(range(5,19)),
    'nnet__hidden_layer_sizes':[30, 40, 50, 60],
    'nnet__activation': ['tanh', 'relu'],
    'nnet__alpha':[0.0001, 0.05]

}
gs = GridSearchCV(pline, param_grid, cv = 5, scoring = 'accuracy')

nested_score = sk.model_selection.cross_val_score(gs,data_x,data_y, cv = 5)

print(nested_score)
print("Accuracy: ", nested_score.mean()*100)

After tuning our hyperparameters, we will now build our final model

In [ ]:
gs.fit(data_x, labels)
best_params = gs.best_params_
print(best_params)

layer_size = best_params.get('nnet__hidden_layer_sizes')
activation_func = best_params.get('nnet__activation')
model_alpha = best_params.get('nnet__alpha')

final_model = Pipeline([('scaling', sk.preprocessing.StandardScaler()), 
                        ('nnet', MLPClassifier(activation = activation_func, 
                                               hidden_layer_sizes=layer_size, 
                                               alpha=model_alpha))])

